# RAG: Retrieval-Augmented Generation


# Caricamento di documenti PDF

## Funzionamento:
1. Si importa la funzione `load_pdfs_recursively` dal modulo `document_loaders` di LangChain.
2. Si definisce la cartella contenente i documenti PDF.
3. Si invoca la funzione `load_pdfs_recursively` passando il percorso della cartella come argomento.
4. La funzione carica ricorsivamente tutti i file PDF da quella cartella e dalle sue sottocartelle.
5. I documenti vengono restituiti come una lista di oggetti `Document`.

In [ ]:
import os
from langchain.document_loaders import PyMuPDFLoader

def load_pdfs_recursively(folder_path):
    """
    Carica ricorsivamente tutti i file PDF da una cartella e dalle sue sottocartelle.

    :param folder_path: Percorso della cartella principale.
    :return: Lista di documenti caricati.
    """
    documents = []

    # Scansiona la cartella e sottocartelle
    for root, _, files in os.walk(folder_path):
        for filename in files:
            print(f"📁 Scansionando: {root}")
            if filename.endswith(".pdf"):
                file_path = os.path.join(root, filename)
                print(f"📄 Caricando: {file_path}")

                # Carica il PDF
                loader = PyMuPDFLoader(file_path)
                documents.extend(loader.load())  # Aggiunge i documenti alla lista

    print(f"✅ Caricati {len(documents)} documenti da {folder_path}")
    return documents

dataset_path = "../HackapizzaDataset/"
all_pdfs = load_pdfs_recursively(dataset_path)


# Caricamento di documenti HTML

## Funzionamento:
1. Si importa la funzione `load_htmls_recursively` dal modulo `document_loaders` di LangChain.
2. Si definisce la cartella contenente i documenti HTML.
3. Si invoca la funzione `load_htmls_recursively` passando il percorso della cartella come argomento.
4. La funzione carica ricorsivamente tutti i file HTML da quella cartella e dalle sue sottocartelle.
5. I documenti vengono restituiti come una lista di oggetti `Document`.

In [50]:
import os
from langchain.document_loaders import BSHTMLLoader

def load_htmls_recursively(folder_path):
    """
    Carica ricorsivamente tutti i file HTML da una cartella e dalle sue sottocartelle.
    """
    documents = []

    for root, _, files in os.walk(folder_path):
        for filename in files:
            if filename.lower().endswith((".html", ".htm")):
                file_path = os.path.join(root, filename)
                print(f"🌍 Caricando: {file_path}")

                try:
                    # Forza l'uso di html.parser se lxml non funziona
                    loader = BSHTMLLoader(file_path, bs_kwargs={"features": "html.parser"})
                    documents.extend(loader.load())
                except Exception as e:
                    print(f"⚠️ Errore con {file_path}: {e}")

    print(f"✅ Caricati {len(documents)} documenti HTML da {folder_path}")
    return documents

dataset_path = "../HackapizzaDataset/"
all_htmls = load_htmls_recursively(dataset_path)

🌍 Caricando: ../HackapizzaDataset/Blogpost/blog_sapore_del_dune.html
🌍 Caricando: ../HackapizzaDataset/Blogpost/blog_etere_del_gusto.html
✅ Caricati 2 documenti HTML da ../HackapizzaDataset/


# Unione di documenti PDF e HTML



In [ ]:
def loadDocuments(folder_path):
    documents = load_htmls_recursively(folder_path)
    documents.extend(load_pdfs_recursively(folder_path))
    return documents

documents = loadDocuments("../HackapizzaDataset/")
print(f"📄 Documenti caricati: {documents}")  # Debug

# Chunking di documenti

## Funzionamento:
Chuking a dimensione fissa di documenti.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = loadDocuments("../HackapizzaDataset/")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)


# Embedding di chunk

## Funzionamento:
Embedding di chunk di testo utilizzando un modello preaddestrato.

In [47]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# Inizializza il modello di embedding
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Genera gli embedding per i chunk di testo
chunk_embeddings = embedding_model.embed_documents([chunk.page_content for chunk in chunks])

# Indicizzamento di chuck in un vettore store

## FAISS

In [48]:
from langchain.vectorstores import FAISS

# Inizializza FAISS con gli embedding e i metadati dei chunk
vectorstore = FAISS.from_texts(
    texts=[chunk.page_content for chunk in chunks],
    embedding=embedding_model
)

# Motore di ricerca



In [54]:
query = "Vorrei assaggiare l'Erba Pipa. In quali piatti la posso trovare?"
results = vectorstore.similarity_search(query, k=10)  # Trova i 5 documenti più simili

# Stampa i risultati
for i, doc in enumerate(results):
    print(f"🔹 Risultato {i+1}: {doc.page_content[:500]}...\n")


🔹 Risultato 1: senza pericolo. Questa base è attraversata dalla Marinatura Psionica, affinata sotto l'attenta supervisione
del Maestro Stellaris, garantendo un'esperienza unica e personalizzata ad ogni assaggio.
La Farina di Nettuno trasforma questo piatto in una sinfonia di leggerezza, dove il pane accompagna il
tutto, perfettamente cotto e con un sentore blu cangiante. Gli Gnocchi del Crepuscolo, saggiamente...

🔹 Risultato 2: Svantaggi: richiede comprensione avanzata dei legami proteici per ottimizzare efficacemente i campi
risonanti senza danno strutturale....

🔹 Risultato 3: del gusto lo ha guidato verso la scoperta di notevoli poteri psionici, permettendogli di intrecciare la trama
della realtà attorno ai suoi piatti. Grazie a questa capacità, egli percepisce i sapori attraverso le menti dei
suoi commensali e crea esperienze gustative che trascendono i limiti dell'immaginazione.
Il suo viaggio lo ha condotto nei deserti scintillanti di Arrakis, dove ha appreso l'arte di manipolar